In [1]:
import torch as t
import numpy as np
import math
from matplotlib import pyplot as plt
from dataclasses import dataclass
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import wandb

In [2]:
@dataclass
class CNNArgs:
    batch_size = 64
    lr = 0.01
    epochs = 3
    img_dim = 28

args = CNNArgs()

In [3]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False,)

In [4]:
class CNN(t.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.layers = t.nn.Sequential(
            t.nn.Conv2d(1, 4, 3, padding='same'),
            t.nn.ReLU(),
            t.nn.MaxPool2d(2, 2),
            t.nn.Flatten(),
            t.nn.Linear(14*14*4, 10),
            # t.nn.ReLU(),
            # t.nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.layers(x)

    def extra_repr(self):
        pass


class FeedForward(t.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.layers = t.nn.Sequential(
            t.nn.Flatten(),
            t.nn.Linear(28*28, 64),
            t.nn.ReLU(),
            t.nn.Linear(64, 10),
            # t.nn.Softmax()
        )

    def forward(self, x):
        return self.layers(x)

    def extra_repr(self):
        pass

In [16]:
model = CNN(args).to('cuda')
optimizer = t.optim.AdamW(model.parameters(), lr=args.lr)

wandb.init(project='trial-sweep') #pass config next; track hyperparams
wandb.watch(models=model, log='all', log_freq=320)

loss_list = []
egs_seen = 0
for imgs, labels in tqdm(train_loader):
    imgs = imgs.to('cuda')
    labels = labels.to('cuda')
    
    logits = model(imgs)
    loss = t.nn.functional.cross_entropy(logits, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    egs_seen += imgs.shape[0]
    wandb.log({'loss': loss.item()}, egs_seen)
    loss_list.append(loss.item())

100%|██████████| 938/938 [00:05<00:00, 169.70it/s]


In [17]:
imgs_correct = 0
imgs_seen = 0

for imgs, labels in tqdm(test_loader):
    imgs = imgs.to('cuda')
    labels = labels.to('cuda')
    
    with t.no_grad():
        logits = model(imgs)
    preds = t.argmax(logits, dim=1)

    assert preds.shape == labels.shape

    imgs_correct += t.sum(preds==labels).item()
    imgs_seen += imgs.shape[0]

accuracy = imgs_correct / imgs_seen
wandb.log({'accuracy':accuracy}, egs_seen)
wandb.finish()

100%|██████████| 157/157 [00:00<00:00, 211.29it/s]


accuracy,▁
loss,█▄▃▂▃▂▁▂▂▂▁▁▂▂▂▁▂▂▂▂▁▁▁▁▂▂▂▂▁▂▁▁▂▂▁▁▁▁▁▁
accuracy,0.9643
loss,0.019


In [18]:
accuracy

0.9643